# Introduction
This notebook is used to experiment with different prompts for the question generation model that uses the chatGPT API from open.ai.
First a helper function is created to call the API with the provided prompt. For this prompt different techniques are tried out and evaluated to find the best performing prompt template.

In [1]:
import os
from dotenv import load_dotenv
import openai
from src.datageneration.extractor import extract_text_without_image
from pypdfium2 import PdfDocument
import pandas as pd
from sklearn.model_selection import train_test_split
from src.evaluation.eval_main import Metrics
import nltk

nltk.download('wordnet')

load_dotenv()
openai.api_key = os.getenv("OPENAI-API-KEY")

def chat_gpt(prompt):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

[nltk_data] Downloading package wordnet to /Users/I516998/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Now the evaluation data is loaded that is also used to evaluate the other question generation models that are used in the Ankinator project.

In [5]:
# # initially retrieve extracted text for each slide - only execute once
slide_path = "../../../datasets/IT-Security_all_slides_no_duplicates.pdf"
pdf = PdfDocument(slide_path)
text = extract_text_without_image(pdf.raw)
extracted_content = pd.DataFrame(columns=['Pagenumber', 'Page-Text', 'OCR-text'])
for i in text:
    extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)

# Define the file path and name
file_path = "../../../datasets/extracted_text_content.csv"

# Save the DataFrame to the specified folder
extracted_content.to_csv(file_path, index=False)

100%|██████████| 596/596 [05:40<00:00,  1.75it/s]
/var/folders/xm/s6jjw11s1vx4n32w6313vgy80000gn/T/ipykernel_69037/3657506144.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)
/var/folders/xm/s6jjw11s1vx4n32w6313vgy80000gn/T/ipykernel_69037/3657506144.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenumber': i[0], 'Page-Text': i[1], 'OCR-text': i[2]}, ignore_index=True)
/var/folders/xm/s6jjw11s1vx4n32w6313vgy80000gn/T/ipykernel_69037/3657506144.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_content = extracted_content.append({'Pagenu

In [2]:
# reload extracted content from file
file_path = "../../../datasets/extracted_text_content.csv"
extracted_content = pd.read_csv(file_path)

In [3]:
file_path = '../../../datasets/Goldstandard.csv'

goldstandard = pd.read_csv(file_path).drop_duplicates()

# Remove unnecessary columns
goldstandard.drop(['PDF-Name', 'Comment','Page Number'], axis=1, inplace=True)

# Join two DataFrames based on index
goldstandard = extracted_content.join(goldstandard, lsuffix='_left', rsuffix='_right')

# Delete records with value "No" and "no" in the "Marked for processing" column
goldstandard = goldstandard[(goldstandard['Marked for processing'] != 'No') & (goldstandard['Marked for processing'] != 'no')]

# Remove unnecessary columns
goldstandard.drop(['Marked for processing', 'Includes Image Data'], axis=1, inplace=True)

# Split the DataFrame into train, validation, and test sets
goldstandard_train_val, goldstandard_test = train_test_split(goldstandard, test_size=0.2, random_state=42)

print("Lenght of test set: ", len(goldstandard_test))
print(goldstandard_test)

Lenght of test set:  93
     Pagenumber                                          Page-Text  \
287         287  Phase 4: Post-Infection\r\n• Carries out attac...   
35           35  • Sig:\r\n– Given a message  ∈ ே and \r\nsecre...   
45           45  Security Engineering\r\n• Aims for engineering...   
283         283  Life Cycle\r\n• The typical life cycle of malw...   
147         147  Related: “Free Model”\r\n• Many services/apps ...   
..          ...                                                ...   
25           25  Key Agreement\r\n• Goal: Establish a common se...   
448         448  HTTP\r\n• HyperText Transfer Protocol (HTTP)\r...   
54           54  Access Matrix\r\nFormal Definition\r\n• At poi...   
216         216  Content Filter\r\n• Content filtering evaluate...   
436         436  DNS Tunneling\r\n• Normal DNS requests only co...   

                                              OCR-text  \
287  te\nPhase 4: Post-Infection Be OP MANNHEIM\n\n...   
35   ol\nExample: R

In [4]:
# Reset the index of the DataFrame
goldstandard_test = goldstandard_test.reset_index(drop=True)
goldstandard_train_val = goldstandard_train_val.reset_index(drop=True)

# this stores now the possible input for the chatGPT model
content = goldstandard_test[["Page-Text", "OCR-text"]]

# this stores the reference
references = goldstandard_test[["Question"]]

In [5]:
references

,Question
0,How is the development phase of the Malware li...
1,Of what are digital signature schemes composed...
2,What is the workflow for MAC?
3,What is an example for a Prime and Probe Attack?
4,What is Baiting?
...,...
88,"What does AES stand for, what key sizes are av..."
89,What is an example for an SQL Injection Attack?
90,What is the Terminology in the context of Acce...
91,What is S/MIME?


In [6]:
content

,Page-Text,OCR-text
0,Phase 4: Post-Infection\r\n• Carries out attac...,te\nPhase 4: Post-Infection Be OP MANNHEIM\n\n...
1,• Sig:\r\n– Given a message ∈ ே and \r\nsecre...,ol\nExample: RSA Signature Scheme BOP MANNHEIM...
2,Security Engineering\r\n• Aims for engineering...,ol\nSecurity Engineering SB OF MANNHEIM\n\n—— ...
3,Life Cycle\r\n• The typical life cycle of malw...,ates\ni UNIVERSITY\nLife Cycle Bor MANNHEIM\n—...
4,Related: “Free Model”\r\n• Many services/apps ...,te\nRelated: “Free Model” Be OP MANNHEIM\n\n— ...
...,...,...
88,Key Agreement\r\n• Goal: Establish a common se...,ol\nKey Agreement Be) OF MANNHEIM\n\n—— School...
89,HTTP\r\n• HyperText Transfer Protocol (HTTP)\r...,HTTP\n\n¢ HyperText Transfer Protocol (HTTP)\n...
90,Access Matrix\r\nFormal Definition\r\n• At poi...,te\nH UNIVERSITY\nAccess Matrix 3? OP MANNHEIM...
91,Content Filter\r\n• Content filtering evaluate...,te\nH NIVERSITY\nContent Filter Be OP MANNHEIM...


# Prompt Engineering
Having prepared everything it is possible to start with prompt engineering. It is started with simple prompts and continued with more complex prompts.
## Zero-Shot
1. Simple prompt

In [11]:
refs = []
for i, q in references.iterrows():
    refs.append((i, [q.item()]))
refs

[(0, ['How is the development phase of the Malware lifecyle defined?']),
 (1, ['Of what are digital signature schemes composed of?']),
 (2, ['What is the workflow for MAC?']),
 (3, ['What is an example for a Prime and Probe Attack?']),
 (4, ['What is Baiting?']),
 (5, ['What is an example for a Heap Overflow Attack?']),
 (6, ['What is the Blockchain?']),
 (7, ['How does a Stack Overflow Attack work?']),
 (8, ['What are possible approaches to define Anonymity?']),
 (9, ['What does Authentication mean?']),
 (10,
  ['What are the exemplary steps of Perturbative Masking through aggregation?']),
 (11, ['How does the process of Signing Transactions work?']),
 (12, ['What does Identification mean?']),
 (13, ['What is the definition of Personal Data?']),
 (14, ['What is Session Hijacking and what are countermeasures? ']),
 (15, ['What are examples of notable XSS Attacks?']),
 (16, ['How are Conflict of Interest Classes defined for Objects?']),
 (17, ['What are common password guidelines?']),
 

In [15]:
model_results = []
# the chatGPT API is called and results are stored
for index, row in content.iterrows():
    if index == 3:
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    ```{row['Page-Text']}```
    """
    model_results.append((index, [chat_gpt(prompt)]))

print(model_results)
# Performance is evaluated
metrics = Metrics(save_to_file=False)
result = pd.DataFrame(
    metrics.evaluate(model_output=model_results, references=refs[:3]),
    index=["ChatGPT"]
)
result

[(0, ['What are some examples of actions taken during the post-infection phase of a cyber attack?']), (1, ['What is the process for computing a signature using the RSA Signature Scheme?']), (2, ['What are the typical aspects to consider when engineering a secure system using a top-down approach in security engineering?'])]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/I516998/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,GRAMMAR,rouge1_fmeasure,rouge1_precision,rouge1_recall,rouge2_fmeasure,rouge2_precision,rouge2_recall,rougeL_fmeasure,rougeL_precision,rougeL_recall,...,min_r,avg_f1,max_f1,min_f1,avg_cos_sim,max_cos_sim,min_cos_sim,avg_sem_meteor,max_sem_meteor,min_sem_meteor
ChatGPT,1,0.191697,0.147115,0.294444,0.027778,0.022222,0.037037,0.191697,0.147115,0.294444,...,0.0,0.188889,0.4,0.0,0.348,0.530378,0.184901,0.262472,0.332237,0.180723


2. Provide more context on the task

In [74]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    When there are examples do not focus on their specifics but try to cover the overarching concept or idea.
    ```{row['Page-Text']}```
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What actions does a virus typically carry out during the post-infection phase?', 'What is the purpose of the "Sig" operation in the RSA Signature Scheme?', 'What does access control in IT security refer to and on what levels can it exist?']


3. Even more context

In [76]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    Focus on concepts, definitions and key-words.
    Take into account how exam questions are normally formulated and formulate the question accordingly.
    When there are examples do not focus on their specifics but try to cover the overarching concept or idea.
    ```{row['Page-Text']}```
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What are some examples of actions that can be carried out during the post-infection phase of a cyberattack?', 'What are the steps involved in the RSA signature scheme?', 'What does access control in IT-Security involve and what are the different levels at which it can exist?']


4. Comparison of copied text and OCR

In [79]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    You are a bot to support in the generation of flashcards from lecture slides.
    You are provided with two inputs. The first input delimited by triple backticks is the text that is copied from the slides.
    The second input delimited by triple quotation marks is retrieved with an OCR tool to extract all text from a slide.
    Follow the below process:
    1. Step: Compare the first input with the second input to retrieve the relevant information
    2. Step: Generate a question for this information in a flashcard style
    Only return the generated question.
    ```{row['Page-Text']}```
    \"\"\"{row['OCR-text']}\"\"\"
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

['What does "Phase 4: Post-Infection" entail?', 'What is the process of generating a signature in the RSA Signature Scheme?', 'What is the purpose of access control in a system?']


## Few-Shot
5. Few-Shot with simple question

In [1]:
model_results = []
# the chatGPT API is called and results are stored
i=0
for index, row in content.iterrows():
    if (i == 3):
        break
    prompt = f"""
    Generate a question in a flashcard style for the content delimited by triple backticks.
    ```{row['Page-Text']}```
    Follow a similar style for generating the question as in this two examples:
    1) Input: {goldstandard_train_val.loc[0, 'Page-Text']}, question: {goldstandard_train_val.loc[0, 'Question']}
    2) Input: {goldstandard_train_val.loc[1, 'Page-Text']}, question: {goldstandard_train_val.loc[1, 'Question']}
    """
    model_results.append(chat_gpt(prompt))
    i=i+1

print(goldstandard_test.loc[1, 'Page-Text'])

print(model_results)
# # Performance is evaluated
# metrics = Metrics(save_to_file=False)
# result = pd.DataFrame(
#     metrics.evaluate(model_output=model_results, references=references),
#     index=["ChatGPT"]
# )
# result

NameError: name 'content' is not defined